# Taxonomic Annotation of Reads

This notebook will go through the workflow assigning taxonomy to reads from a microbiome

1. Taxonomic assignment of reads using Kraken2
2. Refinement of the taxonomic annotation using Bracken


## Getting Started

You will need to rerun this section each time you come back to this notebook to reset all directories and variables.

In [ ]:
# set the variables for your netid and xfile
netid = "MY_NETID"
xfile = "MY_XFILE"

In [ ]:
# Go into the working directory
work_dir = "/xdisk/bhurwitz/bh_class/" + netid + "/assignments/11_taxonomy"
%cd $work_dir

## Creating a config file
Let's create a config file with all of the variables we will need in the scripts below. Then when we want to use these variables in the script, we will "source" the config file to set the variables.

In [ ]:
# create a config file with all of the variables you need
# notice that now we are defining the tools here too, to make out scripts easier to read below
# also, if we want to update to a newer version of the tool we can just edit here.
!echo "export NETID=$netid" > config.sh
!echo "export XFILE=$xfile" >> config.sh
!echo "export WORK_DIR=/xdisk/bhurwitz/bh_class/$netid/assignments/11_taxonomy" >> config.sh
!echo "export XFILE_DIR=/xdisk/bhurwitz/bh_class/$netid/assignments/05_getting_data" >> config.sh
!echo "export FASTQ_DIR=/xdisk/bhurwitz/bh_class/$netid/assignments/07_contam_removal" >> config.sh
!echo "export CONTAINERS=/contrib/singularity/shared/bhurwitz" >> config.sh
!echo "export KRAKEN2=/contrib/singularity/shared/bhurwitz/kraken2:2.1.3--pl5321hdcf5f25_0.sif" >> config.sh
!echo "export BRACKEN=/contrib/singularity/shared/bhurwitz/bracken:2.8--py39h1f90b4d_1.sif" >> conifg.sh
!echo "export KRAKENTOOLS=/contrib/singularity/shared/bhurwitz/krakentools:1.2--pyh5e36f6f_0.sif" >> config.sh
!echo "export DB_DIR=/groups/bhurwitz/databases/kraken2/k2_pluspf_20230605" >> config.sh
!echo "export KMER_SIZE=100" >> config.sh


In [ ]:
# check the config file to be sure it is correct
# Is your netid and xfile correct? Do you have the right directories?
!cat config.sh

## Step 1: Assigning taxonomy to the reads

In this step, we will assign taxonomy to each of the reads in our microbiome (where possible). 

In [ ]:
# Create a script to run bowtie2 to align reads to a human reference
# A few important points:
# 1. We are using the variables from the config file via the `source ./config.sh` command in the script.
# 2. bowtie2 runs on each of the fastq files in the trimmed $FASTQ_DIR
# 3. The results will be written into our $WORK_DIR
# 4. Notice that we are asking for alot more resource (24 cores and 5G of memory per core), we are also asking for more time (24 hours)
my_code = '''#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --nodes=1             
#SBATCH --time=24:00:00   
#SBATCH --partition=standard
#SBATCH --account=bh_class
#SBATCH --array=0-4                         
#SBATCH --output=Job-kraken2-%a.out
#SBATCH --cpus-per-task=24
#SBATCH --mem-per-cpu=5G   

pwd; hostname; date

source $SLURM_SUBMIT_DIR/config.sh
names=($(cat $XFILE_DIR/$XFILE))

SAMPLE_ID=${names[${SLURM_ARRAY_TASK_ID}]}

### reads with human removed to match to the reference database
PAIR1=${FASTQ_DIR}/${SAMPLE_ID}_1.fastq.gz
PAIR2=${FASTQ_DIR}/${SAMPLE_ID}_2.fastq.gz

KRAKEN_OUTDIR=${WORK_DIR}/out_kraken
OUTDIR=${KRAKEN_OUTDIR}/${SAMPLE_ID}
HUMAN_READ_DIR=${OUTDIR}/human_reads
NONHUMAN_READ_DIR=${OUTDIR}/nonhuman_reads

### create the outdir if it does not exist
if [[ ! -d "$KRAKEN_OUTDIR" ]]; then
  echo "$KRAKEN_OUTDIR does not exist. Directory created"
  mkdir $KRAKEN_OUTDIR
fi

if [[ ! -d "$OUTDIR" ]]; then
  echo "$OUTDIR does not exist. Directory created"
  mkdir $OUTDIR
fi

apptainer run ${KRAKEN2} kraken2 \
              --db ${DB_DIR} \ 
              --paired \
              --classified-out ${OUTDIR}/cseqs.fq \
              --output ${OUTDIR}/kraken_results.txt \
              --report ${OUTDIR}/kraken_report.txt \
              --use-names \
              --threads ${SLURM_CPUS_PER_TASK} \ 
              ${PAIR1} ${PAIR2}

TAXID=9606
apptainer run ${KRAKENTOOLS} extract_kraken_reads.py \
             -k ${OUTDIR}/kraken_results.txt \ 
             --report ${OUTDIR}/kraken_report.txt \
             -s1 ${PAIR1} \
             -s2 ${PAIR2} \
             --taxid ${TAXID} \  
             --output ${HUMAN_READ_DIR}/r1.fq \  
             --output2 ${HUMAN_READ_DIR}/r2.fq \
             --include-children \
             --fastq-output

gzip ${HUMAN_READ_DIR}/r1.fq
gzip ${HUMAN_READ_DIR}/r2.fq

### selects all reads NOT from a given set of Kraken taxids (and all children)

apptainer run ${KRAKENTOOLS} extract_kraken_reads.py \
             -k ${OUTDIR}/kraken_results.txt \
             --report ${OUTDIR}/kraken_report.txt \
             -s1 ${PAIR1} \
             -s2 ${PAIR2} \
             --taxid ${TAXID} \
             --output ${NONHUMAN_READ_DIR}/r1.fq \
             --output2 ${NONHUMAN_READ_DIR}/r2.fq \
             --include-children \
             --exclude \
             --fastq-output

gzip ${NONHUMAN_READ_DIR}/r1.fq
gzip ${NONHUMAN_READ_DIR}/r2.fq
 
echo "Finished `date`"

'''

with open('run_kraken_parallel.sh', mode='w') as file:
    file.write(my_code)

In [ ]:
# Check the code and make sure your script above was created.
!cat run_kraken_parallel.sh

In [ ]:
# you should be in your working directory when you run this script
# do you see your config.sh file, and the remove_human_parallel.sh script?
!pwd
!ls

In [ ]:
# Let's run sbatch to run bowtie on each of your trimmed fastq files to remove human
# Remember that this may take a while to run, so take a break, and get a coffee.
!sbatch ./run_kraken_parallel.sh

In [ ]:
# You can check if it is running using the squeue command
# Check for all jobs under your netid
!squeue --user=$netid

In [ ]:
# Once your jobs have run (or are running) you can check the progress
# and also look for errors in the *out files
# For example, you can look at Job-rem_human-0.out
!ls
!cat Job-kraken-0.out

In [ ]:
# Double check that all of your files have run through the human screening
# Do you see a *_1.fastq.gz and *_2.fastq.gz file in the working directory?
# Of not, you will need to check your job out files above for clues about what went wrong.
!ls $work_dir

In [ ]:
# Do the fastq files (post-human filter) look smaller than the ones that were trimmed?
# list the file sizes for fastq files post-human filtering
!ls -l $work_dir

In [ ]:
# list the file sizes for fastq files before human filtering'
!ls -l /xdisk/bhurwitz/bh_class/$netid/assignments/11_taxonomy

Great job! It looks like you have removed *most* of the human reads from your fastq files. We will double check this when we run kraken2 on the files to classify each of the reads by taxonomy.

Another quick note, you may need to remove additional contamination using the same approach. For example, the sequencing center may have use PhiX as a "spike-in" to assess the quality of the sequencing run with a known quantity of DNA. You can use the same approach as above to remove reads from any genome you think may be contaminating your sample.

## Final Step
Copy your notebook to the current working directory

In [ ]:
cp ~/07_contam_removal.ipynb $work_dir